In [1]:
import datetime
import gspread
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from urllib.request import urlopen
import time
from time import sleep
from basketball_reference_scraper.pbp import get_pbp

pd.options.mode.chained_assignment = None  # default='warn'


In [8]:
# read data
nba_results_22_23 = pd.read_csv('22_23_aggregated_results.csv')
nba_results_22_23['DATE'] = pd.to_datetime(nba_results_22_23['DATE'])

In [33]:
nba_results_22_23['AWAY_LP'] = nba_results_22_23['HOME_WP']
nba_results_22_23['HOME_LP'] = nba_results_22_23['AWAY_WP']
nba_results_22_23['HOME_LP'] = nba_results_22_23['AWAY_WP']
nba_results_22_23['TIE_PC'] = 1 - (nba_results_22_23['AWAY_WP'] + nba_results_22_23['HOME_WP'])

In [ ]:
def calculate_metric(df, columns):
    # reformat into single column
    nba_results_22_23_away = nba_results_22_23[['DATE','AWAY_TEAM','AWAY_WP']].rename(columns={'AWAY_TEAM':'TEAM','AWAY_WP':'WP'})
    nba_results_22_23_home = nba_results_22_23[['DATE','HOME_TEAM','HOME_WP']].rename(columns={'HOME_TEAM':'TEAM','HOME_WP':'WP'})
    nba_results_22_23_reformat = pd.concat([nba_results_22_23_away, nba_results_22_23_home]).reset_index(drop=True)
    
    # find team averages
    nba_results_22_23_agg = nba_results_22_23_reformat.groupby(['TEAM']).mean().reset_index()
    
    return nba_results_22_23_agg
    

In [16]:
# reformat into single column
nba_results_22_23_away = nba_results_22_23[['DATE','AWAY_TEAM','AWAY_WP']].rename(columns={'AWAY_TEAM':'TEAM','AWAY_WP':'WP'})
nba_results_22_23_home = nba_results_22_23[['DATE','HOME_TEAM','HOME_WP']].rename(columns={'HOME_TEAM':'TEAM','HOME_WP':'WP'})
nba_results_22_23_reformat = pd.concat([nba_results_22_23_away, nba_results_22_23_home]).reset_index(drop=True)

In [24]:
# find team averages
nba_results_22_23_agg = nba_results_22_23_reformat.groupby(['TEAM']).mean().reset_index()
# nba_results_22_23_agg.sort_values('WP',ascending=False).reset_index(drop=True)

In [25]:
## final df
## what else do i want....
### team
### winning time percent
### losing time percent
### tied-time percent
### Actual WP%
### Point Differential
### Delta: Winning Time Percent vs Actual WP%
### Delta: Winning Time Percent vs Point Diff (HOW DO I DO THIS)
nba_results_22_23_agg

,TEAM,WP
0,ATL,0.512354
1,BOS,0.627872
2,BRK,0.488815
3,CHI,0.466179
4,CHO,0.336131
5,CLE,0.532688
6,DAL,0.456318
7,DEN,0.547795
8,DET,0.268981
9,GSW,0.477099
